## Robot Framework kernel for Jupyter Lite

This notebook demonstrates the use of Robot Framework kernel for in-browser Jupyter Lite.

At first we import `IPython.display` to have its display method as a keyword.

In [ ]:
*** Settings ***

Library  IPython.display

Then we try out successful task that produces log and report:

In [ ]:
*** Tasks ***

This task shall pass
    Should be equal  1  1

So, happy path works. Let's see next how the failing task also shows us stderr from the execution:

In [ ]:
*** Tasks ***

This task shall fail
    Should be equal  1  2

It is also possible to define custom keywords. And make them require arguments.

Robot Framework kernel injects widgets for the keywords to make it easier to try them out:

In [ ]:
*** Keywords ***

This keyword displays a word
    [Arguments]  ${word}=Hello world
    Display  ${word}

Unfortunately, due to [a bug](https://github.com/jupyterlite/jupyterlite/issues/303) in Jupyter Lite ipywidgets integration, the button output does not fully work: output may replace button or appear in wrong location.

Also, the failing keyword below does not show its stderr as it should:

In [ ]:
*** Keywords ***

This keyword should fail
    [Arguments]  ${word}=Hello world
    Don't display  ${word}

We expect these to be fixed pretty soon...

Next, to the fancier features. Let's define inline Python module for custom Python keyword library:

In [ ]:
%%python module HelloWorld

class HelloWorld:
    def hello_world(self):
        return "Hello World!"

Now this module can be safely imported as a keyword library:

In [ ]:
*** Settings ***

Library  HelloWorld

And used in a task:

In [ ]:
*** Tasks ***

Try out HelloWorld keywords
    Hello world

Adding existing 3rd party keyword libraries is more tricky, though.

At first, in-browser Jupyter Lite can only support pure Python packages (or packages pre-compiled to WASM). At second, Robot Framework kernel in this integration does not expose API to interact with the underlying Python environment.

A workaround is to define a custom in-line module and import it twice by executing the cell. The first execution will trigger asynchronous package download. Once the package has been downloaded, execution should work.

In [ ]:
%%python module PackageManager

try:
    import OTP
except ImportError:
    import micropip
    micropip.install(["pyotp", "robotframework-otp"])
    import OTP

In [ ]:
*** Settings ***

Library  OTP

In [ ]:
*** Tasks ***

Get OTP from secret
    ${otp}=  Get OTP  base32secret3232

Finally, we learn to interact with the world from in-browser Jupyter Lite. Because the underlying Python implementation does not yet have `http.client`, we can only do request with JavaScript API, only using synchronous version of XMLHttpRequest, only for resources that support CORS.

In [ ]:
%%python module API

from js import XMLHttpRequest

import json


class API:
    def get_data(self):
        url = f"https://reqres.in/api/users"
        request = XMLHttpRequest.new()
        request.open("GET", url, False)
        request.send(None)
        assert request.status == 200
        return json.loads(request.responseText)

In [ ]:
*** Settings ***

Library  API

In [ ]:
*** Tasks ***

Get some data
    ${data}=  Get data